In [ ]:
#Optimization Example

In [42]:
import geopandas as gpd
import pandas as pd
import os, sys
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import LoadOSM as losm
import importlib
from pulp import LpInteger,LpVariable, LpProblem, lpSum, LpMinimize

In [44]:
osmNetwork = losm.OSM_to_network('./sampleData/nouakchott/mauritania-latest.osm.pbf')

In [45]:
osmNetwork.roads_raw.infra_type.value_counts()

residential       37459
track              5467
unclassified       4021
service            2028
path               1488
tertiary            891
footway             563
secondary           310
primary             228
construction        105
trunk                83
pedestrian           29
trunk_link           29
tertiary_link        14
primary_link          5
secondary_link        3
road                  2
raceway               2
proposed              2
cycleway              1
Name: infra_type, dtype: int64

In [46]:
accepted_road_types = ['residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link']

In [47]:
osmNetwork.filterRoads(acceptedRoads = accepted_road_types)

In [48]:
shp = gpd.read_file('./sampleData/nouakchott/layers/POLYGON.shp')
shp = shp.to_crs({'init':'epsg:4326'})
shp_obj = shp.geometry.iloc[0]

In [49]:
osmNetwork.roads_raw = osmNetwork.roads_raw.loc[osmNetwork.roads_raw.geometry.intersects(shp_obj) == True]

In [50]:
osmNetwork.generateRoadsGDF(verbose = False)

In [51]:
osmNetwork.initialReadIn()

In [52]:
gn.save(osmNetwork.network,'nouakchott','./sampleData/nouakchott/')

In [53]:
#the cleaning network part
import os, sys, time
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

In [54]:
def CleanNetwork(G, wpath, country, UTM, WGS = {'init': 'epsg:4326'}, junctdist = 50, verbose = False):
    
    ### Topologically simplifies an input graph object by collapsing junctions and removing interstital nodes
    # REQUIRED - G: a graph object containing nodes and edges. edges should have a property 
    #               called 'Wkt' containing geometry objects describing the roads
    #            wpath: the write path - a drive directory for inputs and output
    #            country: this parameter allows for the sequential processing of multiple countries
    #            UTM: the epsg code of the projection, in metres, to apply the junctdist
    # OPTIONAL - junctdist: distance within which to collapse neighboring nodes. simplifies junctions. 
    #            Set to 0.1 if not simplification desired. 50m good for national (primary / secondary) networks
    #            verbose: if True, saves down intermediate stages for dissection
    ################################################################################################
    
    # Squeezes clusters of nodes down to a single node if they are within the snapping tolerance
    a = gn.simplify_junctions(G, UTM, WGS, junctdist)

    # ensures all streets are two-way
    a = gn.add_missing_reflected_edges(a)
    
    #save progress
    if verbose is True: 
        gn.save(a, 'a', wpath)
    
    # Finds and deletes interstital nodes based on node degree
    b = gn.custom_simplify(a)
    
    # rectify geometry
    for u, v, data in b.edges(data = True):
        if type(data['Wkt']) == list:
                data['Wkt'] = gn.unbundle_geometry(data['Wkt'])
    
    # save progress
    if verbose is True: 
        gn.save(b, 'b', wpath)
    
    # For some reason CustomSimplify doesn't return a MultiDiGraph. Fix that here
    c = gn.convert_to_MultiDiGraph(b)

    # This is the most controversial function - removes duplicated edges. This takes care of two-lane but separate highways, BUT
    # destroys internal loops within roads. Can be run with or without this line
    c = gn.remove_duplicate_edges(c)

    # Run this again after removing duplicated edges
    c = gn.custom_simplify(c)

    # Ensure all remaining edges are duplicated (two-way streets)
    c = gn.add_missing_reflected_edges(c)
    
    # save final
    gn.save(c, '%s_processed' % country, wpath)
    
    print('Edge reduction: %s to %s (%d percent)' % (G.number_of_edges(), 
                                               c.number_of_edges(), 
                                               ((G.number_of_edges() - c.number_of_edges())/G.number_of_edges()*100)))
    return c

In [55]:
UTMZs = {'MRT':32628}

WGS = {'init': 'epsg:4326'}

countries = ['MRT']

wpath = r'./sampleData/nouakchott/'

for country in countries:
    
    print('\n--- processing for: %s ---\n' % country)
    print('start: %s\n' % time.ctime())

    print('Outputs can be found at: %s\n' % (wpath))
        
    UTM = {'init': 'epsg:%d' % UTMZs[country]}
    
    G = nx.read_gpickle(os.path.join(wpath, 'nouakchott.pickle'))
    
    G = CleanNetwork(G, wpath, country, UTM, WGS, 0.5, verbose = False)
    print('\nend: %s' % time.ctime())
    print('\n--- processing complete for: %s ---' % country)


--- processing for: MRT ---

start: Fri Mar 22 11:20:41 2019

Outputs can be found at: ./sampleData/nouakchott/

60139
120238
118392
59249
118388
Edge reduction: 60142 to 118388 (-96 percent)

end: Fri Mar 22 11:27:43 2019

--- processing complete for: MRT ---


In [14]:
#net prep phase

In [56]:
#read back in processed graph
G = nx.read_gpickle('./sampleData/nouakchott/MRT_processed.pickle')

In [57]:
G.number_of_edges()

118388

In [58]:
G.number_of_nodes()

37305

In [59]:
sd = {          'residential': 10,  # kmph
                'primary': 25, # kmph
                'primary_link':20,
                'motorway':35,
                'motorway_link': 25,
                'trunk': 20,
                'trunk_link':15,
                'secondary': 10, # kmph
                'secondary_link':5,
                'tertiary':5,
                'tertiary_link': 5,
                'unclassified':5
                }

In [60]:
gn.example_edge(G)

(0, 24505, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 0.03882328422750167, 'Type': 'legitimate'})


In [61]:
#look into modifying this for weighted distances
G_time = gn.convert_network_to_time(G, 
                                   distance_tag = 'length',
                                   graph_type = 'drive', 
                                   road_col = 'infra_type',
                                   speed_dict = sd, 
                                   factor = 1000)

In [62]:
gn.example_edge(G_time)

(0, 24505, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 38.82328422750167, 'Type': 'legitimate', 'time': 13.976382321900601, 'mode': 'drive'})


In [63]:
D = list(nx.strongly_connected_component_subgraphs(G_time))

In [64]:
len(D)

11

In [65]:
G = D[0]

In [66]:
G = nx.convert_node_labels_to_integers(G)

In [67]:
gn.save(G, 'biggest_subg', './sampleData/nouakchott/')

In [68]:
#make sure origin and destination are created. They will be CSVs with 'Lat' and 'Lon' columns.
#There will also need to be a way in the future to add a 'demand' column for the origins CSV, and a 'capacity' column for the destinations dataset.
origins = pd.read_csv('./sampleData/nouakchott/origins_test1.csv')


In [ ]:
origins['geometry'] = list(zip(origins['Lon'],origins['Lat']))
origins['geometry'] = origins['geometry'].apply(Point)
origins_gdf = gpd.GeoDataFrame(origins, crs = {'init':'epsg:4326'}, geometry = 'geometry')
origins_gdf = gn.pandana_snap(G, origins_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
origins = list(origins_gdf.NN)
origins = list(set(origins))

In [ ]:
destinations = pd.read_csv('./sampleData/nouakchott/destinations_test1.csv')

In [ ]:
destinations['geometry'] = list(zip(destinations['Lon'],destinations['Lat']))
destinations['geometry'] = destinations['geometry'].apply(Point)
destinations_gdf = gpd.GeoDataFrame(destinations, crs = {'init':'epsg:4326'}, geometry = 'geometry')
destinations_gdf = gn.pandana_snap(G, destinations_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
#destinations_gdf.NN is the nearest node of the road network
destinations = list(destinations_gdf.NN)
destinations = list(set(destinations))

In [ ]:
destinations_gdf

In [ ]:
print("length of origins is %s" % len(origins))

In [ ]:
print("length of destinations is %s" % len(destinations))

In [ ]:
len(origins) * len(destinations)

In [34]:
d_test = destinations[:50]
print(d_test)

[8994, 7011, 6724, 4965, 2440, 1646, 25486, 33112, 15739, 4092]


In [35]:
#look more into this calculate_OD function
%time OD = gn.calculate_OD(G, origins, destinations, fail_value = 9999999999999)

CPU times: user 2.51 s, sys: 0 ns, total: 2.51 s
Wall time: 3 s


In [36]:
origins_df = pd.DataFrame(OD, columns = destinations, index = origins)

In [37]:
facilities = origins_df.columns.values.tolist()

In [38]:
facilities

[8994, 7011, 6724, 4965, 2440, 1646, 25486, 33112, 15739, 4092]

In [39]:
result = gn.optimize_facility_locations(origins_df, facilities, 4, existing_facilities = None)

In [40]:
result

[15739, 2440, 4092, 7011]

In [ ]:
#result is 5,9,7,10